In [1]:
import numpy as np
import pandas as pd 
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import re
import sys, os, re, csv, codecs, numpy as np, pandas as pd
from tensorflow.keras.layers import GRU
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers


from tensorflow.keras.layers import Input, Dense,multiply

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K



class Attention(tf.keras.layers.Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        """
        Keras Layer that implements an Attention mechanism for temporal data.
        Supports Masking.
        Follows the work of Raffel et al. [https://arxiv.org/abs/1512.08756]
        # Input shape
            3D tensor with shape: `(samples, steps, features)`.
        # Output shape
            2D tensor with shape: `(samples, features)`.
        :param kwargs:
        Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
        The dimensions are inferred based on the output shape of the RNN.
        Example:
            model.add(LSTM(64, return_sequences=True))
            model.add(Attention())
        """
        self.supports_masking = True
        #self.init = initializations.get('glorot_uniform')
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight(shape=(input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight(shape=(input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        # eij = K.dot(x, self.W) TF backend doesn't support it

        # features_dim = self.W.shape[0]
        # step_dim = x._keras_shape[1]

        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)), K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
    #print weigthted_input.shape
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]
        return input_shape[0],  self.features_dim


In [4]:
prefix = '../Datasets/'
train_df = pd.read_csv(prefix + 'train_for_db.tsv', sep='\t')
test_df = pd.read_csv(prefix + 'test_for_db.tsv', sep='\t')
eval_df = pd.read_csv(prefix + 'val_for_db.tsv', sep='\t')
train_df = train_df.dropna()
test_df = test_df.dropna()
eval_df = eval_df.dropna()

In [5]:
X_train = np.array(train_df["text"])
Y_train = np.array(train_df["feedback_realtor"]).reshape((-1, 1))
X_test = np.array(eval_df["text"])
Y_test = np.array(eval_df["feedback_realtor"]).reshape((-1, 1))
print(X_train.shape)
print(X_test.shape)

(625410,)
(78176,)


In [6]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    print("Running on TPU ", tpu.cluster_spec().as_dict()["worker"])
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()
print("REPLICAS: ", strategy.num_replicas_in_sync)

REPLICAS:  1


In [7]:
# Text Vectors
MAX_FEATURES = 70000
EMBED_DIM = 300

# LSTM
LSTM_OUTPUT_SIZE = 200
MAXLEN = 200

# Training
BATCH_SIZE = 256* strategy.num_replicas_in_sync
EPOCHS = 10

In [8]:
tokenizer = Tokenizer(lower=True, split=" ", num_words=MAX_FEATURES)
tokenizer.fit_on_texts(X_train)
X_train_vec = tokenizer.texts_to_sequences(X_train)
X_test_vec = tokenizer.texts_to_sequences(X_test)
#max([len(x) for x in X_train_vec])
print(f"Max vector length: {MAXLEN}")

Max vector length: 200


In [9]:
from tensorflow.keras.preprocessing import sequence

In [10]:
#pad with zeros for same vector length
X_train_vec = sequence.pad_sequences(X_train_vec, maxlen=MAXLEN, padding="post")
X_test_vec = sequence.pad_sequences(X_test_vec, maxlen=MAXLEN, padding="post")

In [11]:
X_test_final = np.array(test_df["text"])
Y_test_final = np.array(test_df["feedback_realtor"]).reshape((-1, 1))

tokenizer = Tokenizer(lower=True, split=" ", num_words=MAX_FEATURES)
tokenizer.fit_on_texts(X_test_final)
X_test_final_vec = tokenizer.texts_to_sequences(X_test_final)
MAXLEN = 200
#max([len(x) for x in X_train_vec])
print(f"Max vector length: {MAXLEN}")
X_test_final = sequence.pad_sequences(X_test_final_vec, maxlen=MAXLEN, padding="post")

Max vector length: 200


In [12]:
PATH_DATA = './outputs_new/lstm_attention'
PATH_MODELS = PATH_DATA + "model_pre_trained/"
PATH_CHECKPOINTS = PATH_MODELS + "checkpoints/"

In [13]:
opt_adam = tf.keras.optimizers.Adam(
    learning_rate=0.001)

In [14]:
# Define NN architecture
embedding_layer = Embedding(MAX_FEATURES,
        EMBED_DIM,
        input_length = 200)

lstm_layer = LSTM(LSTM_OUTPUT_SIZE, dropout=0.1, recurrent_dropout=0.1,return_sequences=True)
text_input = Input(shape=(200,), dtype='int32')
embedded_sequences= embedding_layer(text_input)
x = lstm_layer(embedded_sequences)
merged = Attention(200)(x)
preds = Dense(1, activation='sigmoid')(merged)
model = Model(inputs=[text_input], \
        outputs=preds)

model.compile(loss='binary_crossentropy',
        optimizer=opt_adam,
        metrics=['accuracy', 'AUC'])
print(model.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 200)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 200, 300)          21000000  
_________________________________________________________________
lstm (LSTM)                  (None, 200, 200)          400800    
_________________________________________________________________
attention (Attention)        (None, 200)               400       
_________________________________________________________________
dense (Dense)                (None, 1)                 201       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 2         
Total params: 21,401,403
Trainable params: 21,401,403
Non-trainable params: 0
_________________________________________________

In [17]:
def get_config(self):
        config = super().get_config().copy()
        config.update({
            'vocab_size': self.vocab_size,
            'num_layers': self.num_layers,
            'units': self.units,
            'd_model': self.d_model,
            'num_heads': self.num_heads,
            'dropout': self.dropout,
        })
        return config

In [ ]:
%%time
# Stop training when validation acc starts dropping
# Save checkpoint of model each period
now = datetime.now().strftime("%Y-%m-%d_%H%M")
# Create callbacks
callbacks = [
    EarlyStopping(monitor="val_loss", verbose=1, patience=2),
    ModelCheckpoint(
        PATH_CHECKPOINTS + now + "_Model_BILSTM-Embed_.h5",
        monitor="val_loss",
        save_best_only=True,
        verbose=1,
    ),
]

# Fit the model
steps_per_epoch = int(np.floor((len(X_train_vec) / BATCH_SIZE)))
print(
    f"Model Params.\nbatch_size: {BATCH_SIZE}\nEpochs: {EPOCHS}\n"
    f"Step p. Epoch: {steps_per_epoch}\n"
)

hist = model.fit(
    X_train_vec,
    Y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    steps_per_epoch=steps_per_epoch,
    callbacks=callbacks,
    validation_data=(X_test_vec, Y_test),
)

In [18]:
model.save('model_lstm_attention_150721')

INFO:tensorflow:Assets written to: model_lstm_attention_150721/assets


In [19]:
# Predict on test data
prediction = model.predict(X_test_final)

In [21]:
prediction = pd.DataFrame(prediction)

In [22]:
prediction.to_csv('prediction_lstm_attention.csv', sep = ',')